<a href="https://colab.research.google.com/github/kartikayladdha/BDT/blob/main/Lab%205/%20BigData_Lab_5_PySpark_Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Vidhi Kapoor - J021**
#**Kartikay Laddha - J025**

##  PySpark setup

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/nul
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

## Download movie lens data:  
https://www.kaggle.com/grouplens/movielens-20m-dataset?select=movie.csv

In [3]:
! pip install -q kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"vidhikapoor","key":"92b7db479a796a8dd0c55549d3f749ef"}'}

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              222KB  2021-04-05 06:19:09           2067  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44           1221  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            946  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19            444  
promptcloud/care

In [6]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip /content/movielens-20m-dataset.zip

Archive:  /content/movielens-20m-dataset.zip
replace genome_scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: genome_scores.csv       
replace genome_tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: genome_tags.csv         
replace link.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: link.csv                
replace movie.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: movie.csv               
replace rating.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rating.csv              y

replace tag.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: tag.csv                 


# Movie Recommendation using PySpark

In [8]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [9]:
from pyspark.sql.functions import col

In [10]:
from pyspark.sql.functions import lit

In [11]:
def get_mat_sparsity(ratings):
    # Count the total number of ratings in the dataset
    count_nonzero = ratings.select("rating").count()

    # Count the number of distinct userIds and distinct movieIds
    total_elements = ratings.select("userId").distinct().count() * ratings.select("movieId").distinct().count()

    # Divide the numerator by the denominator
    sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
    print("The ratings dataframe is ", "%.2f" % sparsity + "% sparse.")

In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendations').getOrCreate()
movies = spark.read.csv("movie.csv",header=True)
ratings = spark.read.csv("rating.csv",header=True)
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|     4|2004-09-10 03:08:54|
|     1|    223|     4|2005-04-02 23:46:13|
|     1|    253|     4|2005-04-02 23:35:40|
|     1|    260|     4|2005-04-02 23:33:46|
|     1|    293|     4|2005-04-02 23:31:43|
|     1|    296|     4|2005-04-02 23:32:47|
|     1|    318|     4|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|     4|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|     3|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

In [13]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [14]:
ratings = ratings.\
          withColumn('userId', col('userId').cast('integer')).\
          withColumn('movieId', col('MovieId').cast('integer')).\
          withColumn('rating', col('rating').cast('float')).\
          drop('timestamp')

ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
|     1|    293|   4.0|
|     1|    296|   4.0|
|     1|    318|   4.0|
|     1|    337|   3.5|
|     1|    367|   3.5|
|     1|    541|   4.0|
|     1|    589|   3.5|
|     1|    593|   3.5|
|     1|    653|   3.0|
|     1|    919|   3.5|
+------+-------+------+
only showing top 20 rows



In [15]:
movie_ratings = ratings.join(movies, ['movieId'], 'left')

In [16]:
get_mat_sparsity(ratings)

The ratings dataframe is  99.46% sparse.


In [17]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

In [ ]:
# def get_binary_data(ratings):
#   ratings = ratings.withColumn('binary', lit(1))
#   userIds = ratings.select('userId').distinct()
#   movieIds = ratings.select('movieId').distinct()

#   user_movie = userIds.crossJoin(movieIds).join(ratings, ['userId', 'movieId'], "left")
#   user_movie = user_movie.select(['userId', 'movieid', 'binary']).fillna(0)
#   return user_movie

# user_movie = get_binary_data(ratings)
# user_movie

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [19]:
als = ALS(
            userCol = 'userId',
          itemCol = 'movieId',
          ratingCol = 'rating',
          nonnegative = True,
          implicitPrefs = False,
          coldStartStrategy = 'drop'

)

In [20]:
type(als)

pyspark.ml.recommendation.ALS

In [21]:
param_grid = ParamGridBuilder()\
              .addGrid(als.rank,[50])\
              .addGrid(als.regParam,[.15])\
              .build()



evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol= 'prediction')

print(f'Total models = {len(param_grid)}')

Total models = 1


In [22]:
cv = CrossValidator(estimator=als,
                    estimatorParamMaps=param_grid,
                    evaluator = evaluator,
                    numFolds=5)

In [23]:
train.dtypes

[('userId', 'int'), ('movieId', 'int'), ('rating', 'float')]

In [24]:
model = cv.fit(train)
best_model = model.bestModel

In [25]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

print("  Rank:", best_model._java_obj.parent().getRank())

print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


In [26]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8414607822843814


In [27]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|136222|    148|   2.0| 2.4865942|
|137949|    148|   4.0| 3.0610926|
| 88527|    148|   2.0| 2.4003563|
| 96427|    148|   3.0| 2.9490657|
| 36821|    148|   4.0| 2.9261234|
| 32882|    148|   3.0|  2.803288|
| 46146|    148|   2.0| 1.9056679|
| 54726|    148|   5.0| 3.2069943|
|  3990|    148|   4.0|  2.396212|
| 80693|    148|   1.0|  2.821439|
| 64843|    148|   3.5| 2.7011235|
| 81300|    148|   1.0| 2.8317454|
| 48644|    148|   3.0|  2.974217|
| 62028|    148|   3.0|  2.751502|
|128653|    148|   2.0| 2.4537754|
|  9084|    148|   2.0| 2.9348912|
| 70446|    148|   2.0| 2.9708045|
| 75359|    148|   3.0| 3.0554786|
|110991|    148|   2.5| 2.7469063|
| 10434|    148|   3.0| 2.5916421|
+------+-------+------+----------+
only showing top 20 rows



In [28]:
from pyspark.sql.functions import explode

In [29]:
recommendations = best_model.recommendForAllUsers(5)
recommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[{126219, 8.53115...|
|   463|[{126219, 8.27961...|
|   471|[{126219, 7.53670...|
|   496|[{124273, 8.27547...|
|   833|[{126219, 7.04592...|
|  1088|[{126219, 6.46081...|
|  1238|[{126219, 7.27851...|
|  1342|[{124273, 8.41849...|
|  1580|[{126219, 5.49291...|
|  1591|[{126219, 8.04896...|
|  1645|[{126219, 7.71992...|
|  1829|[{126219, 8.06880...|
|  1959|[{124273, 5.78672...|
|  2122|[{124273, 5.68641...|
|  2142|[{124273, 7.66215...|
|  2366|[{126219, 8.15004...|
|  2659|[{124273, 7.83407...|
|  2866|[{124273, 7.44986...|
|  3175|[{126219, 8.36163...|
|  3749|[{124273, 7.70656...|
+------+--------------------+
only showing top 20 rows



In [31]:
nrecommendations = recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))
nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|   148| 126219| 8.531154|
|   148| 124273|6.4166775|
|   148| 121029| 5.992231|
|   148| 107252| 5.394761|
|   148| 114906|5.3100457|
|   463| 126219| 8.279614|
|   463| 124273|7.6671324|
|   463| 121029|6.4955397|
|   463| 107252|5.9404917|
|   463| 110173|5.7849936|
+------+-------+---------+



In [32]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+-----------------+
|movieId|userId|   rating|               title|           genres|
+-------+------+---------+--------------------+-----------------+
| 124273|   100| 6.972379|Kevin Smith: Too ...|           Comedy|
| 126219|   100|6.0728464|    Marihuana (1936)|Documentary|Drama|
| 121029|   100|5.3636446|No Distance Left ...|      Documentary|
|  81117|   100|  5.12783|Moth, The (Cma) (...|            Drama|
| 130347|   100|4.9169455|Bill Hicks: Sane ...|           Comedy|
+-------+------+---------+--------------------+-----------------+



In [33]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|     50|   100|   5.0|Usual Suspects, T...|Crime|Mystery|Thr...|
|    293|   100|   5.0|Léon: The Profess...|Action|Crime|Dram...|
|    680|   100|   5.0|Alphaville (Alpha...|Drama|Mystery|Rom...|
|   1449|   100|   5.0|Waiting for Guffm...|              Comedy|
|    235|   100|   4.0|      Ed Wood (1994)|        Comedy|Drama|
|    162|   100|   4.0|        Crumb (1994)|         Documentary|
|    223|   100|   4.0|       Clerks (1994)|              Comedy|
|    260|   100|   4.0|Star Wars: Episod...|Action|Adventure|...|
|    265|   100|   4.0|Like Water for Ch...|Drama|Fantasy|Rom...|
|    288|   100|   4.0|Natural Born Kill...|Action|Crime|Thri...|
+-------+------+------+--------------------+--------------------+

